In [ ]:
# Load in software packages
%matplotlib inline
from parcels import DiffusionUniformKh, FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, AdvectionEE, plotTrajectoriesFile, Variable
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

In [ ]:
filenames = {'U':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'V':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc",
             'bathy':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'mask':'chapter_4_models/copernicus_phy001_030_Statics/GLO-MFC_001_030_mask_bathy.nc',
             'S':"chapter_4_models/copernicus_phy001_030/global-reanalysis-phy-001-030-daily_*.nc"}
variables = {'U':'uo',
             'V':'vo',
             'S':'so',
             #'mask':'mask',
            'bathy':'deptho'}
dimensions = {'U': {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'},
              'V': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'S': {'lon': 'longitude', 'lat': 'latitude','time': 'time'},
              'mask': {'lon': 'longitude', 'lat': 'latitude', 'depth':'depth'},
              'bathy': {'lon': 'longitude', 'lat': 'latitude'}}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation = True, mesh='spherical', deferred_load=True)

In [ ]:
#Kh_zonal = 100
#Kh_meridional = 100
#
#fieldset.add_constant_field("Kh_zonal", Kh_zonal, mesh='spherical')
#fieldset.add_constant_field("Kh_meridional", Kh_meridional, mesh='spherical')

In [ ]:
class KelpParticle(ScipyParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    VelU = Variable('VelU', dtype=np.float32) # create variable for meridonal velocity
    VelV = Variable('VelV', dtype=np.float32) # create variable for zonal velocity
    WindU = Variable('WindU')
    WindV = Variable('WindV')
    KelpVelU = Variable('KelpVelU')
    KelpVelV = Variable('KelpVelV')
    Phase = Variable('Phase', dtype=np.float32, initial = 1) # 1-floating, 2-beached, 3-out of bounds
    waterdragU = Variable('waterdragU')
    waterdragV = Variable('waterdragV')
    winddragU = Variable('winddragU')
    winddragV = Variable('winddragV')
    stokesdragU = Variable('stokesdragU')
    stokesdragV = Variable('stokesdragV')
    Salinity = Variable('Salinity', initial = fieldset.S) # create variable for salinity
    #Mask = Variable('Mask', initial = fieldset.mask, to_write = False) # create variable for the LandMask


def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat
    
# Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        particle.VelU = fieldset.U[time, particle.depth, particle.lat, particle.lon]
        particle.VelV = fieldset.V[time, particle.depth, particle.lat, particle.lon]
        particle.Salinity = fieldset.S[time, particle.depth, particle.lat, particle.lon] # partical salinity units
        so = ((particle.Salinity/1000)*0.001) # conversion of salinity to correct units
        Ac =  310.54  # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65  # total mass of the kelp plant being simulated
        particle.waterdragU = ((0.5*so*(particle.VelU*particle.VelU)*0.47*Ac))/mass # syntax for calculating the loss in velocity as a result of drag
        particle.waterdragV = ((0.5*so*(particle.VelV*particle.VelV)*0.47*Ac))/mass
        particle.KelpVelU = particle.VelU-particle.waterdragU
        particle.KelpVelV = particle.VelV-particle.waterdragV
        particle.lon  += particle.KelpVelU*particle.dt  # overall velocity minus the loss in the velocity as a result of drag
        particle.lat  += particle.KelpVelV*particle.dt  # overall velocity minus the loss in the velocity as a result of drag
        particle.Phase = 1
                
def Age(particle, fieldset, time):
        particle.age += (particle.dt/3600)
        
#def Beaching(particle, fieldset, time):
#    if particle.Phase == 1:
#        particle.Mask = fieldset.mask[time, particle.depth,particle.lat, particle.lon]
#        if particle.Mask <= 0.5:
#            particle.Phase = 4
#            particle.delete()
                    
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()
    particle.Phase = 2       

In [ ]:
# Year1 Summer: 2017, 12, 1
# Year1 Autumn: 2018, 3, 1
# Year1 Winter: 2018, 6, 1
# Year1 Spring: 2018, 9, 1

pset = ParticleSet.from_line(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=KelpParticle,  # the type of particles (JITParticle/ScipyParticle/Custom)
                             start=[18.3176, -34.1501], 
                             finish=[9.0000, -34.1501],
                             size = 10,
                             repeatdt = 10800,
                             time=(datetime(2018, 3, 1)))   

In [ ]:
## cast kernels, select appropriate 'kernel set'

#kernels = pset.Kernel(AdvectionRK4) + SampleDistance + WaterDrag + Age #+ Beaching # KelpFloat
kernels = pset.Kernel(SampleDistance) + WaterDrag + Age #+ DiffusionUniformKh #+ Beaching # KelpFloat


In [ ]:
## execute simulation

## define output file, select outfile according to simulation type

pset.execute(kernels, 
             runtime=timedelta(days=91),
             dt=timedelta(hours=3),
             output_file=pset.ParticleFile(name="exp2_kelp_hydro_autumn_rep.nc", outputdt=timedelta(hours=3)), 
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
# simple plot for viewing, use for checking simulation output
plotTrajectoriesFile('exp2_kelp_hydro_test.nc')
plotTrajectoriesFile('exp2_kelp_hydro_summer.nc')
plotTrajectoriesFile('exp2_kelp_hydro_test.nc')

In [ ]:
plotTrajectoriesFile('exp2_kelp_hydro_test.nc', mode='movie2d_notebook')  

In [ ]:
plotTrajectoriesFile('exp2_passive_summer.nc', mode='movie2d_notebook')  

In [ ]:
pset.show(field='vector', show_time=datetime(2018, 1, 30, 0), domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day30', with_particles = False, vmax=0.8)